In [1]:
import os
import sys
sys.path.append('../modules')

import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = InteractiveSession(config=config)

from srgan_d import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
r = 2
input_dir = f'../data/train/{r}/low/'
tgt_dir = f'../data/train/{r}/high/'

input_dir_val = f'../data/val/{r}/low/'
tgt_dir_val = f'../data/val/{r}/high/'


test_file = 'UBMk30rjy0o_17675_42.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_dir = '../data/out/'

model_dir = '../models/'
vgg_name = 'vgg16_notop.hdf5'
vgg_path = model_dir + vgg_name

input_shape = (67, 120, 3)
tgt_shape = (134, 240, 3)

batch_size = 8

In [3]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

../modules/srgan_d.py:111: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
../modules/srgan_d.py:153: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
srgan.generator.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 67, 120, 3)   0                                            
__________________________________________________________________________________________________
G_Head_conv0 (Conv2D)           (None, 67, 120, 128) 3584        input_1[0][0]                    
__________________________________________________________________________________________________
G_Head_norm0 (BatchNormalizatio (None, 67, 120, 128) 512         G_Head_conv0[0][0]               
__________________________________________________________________________________________________
G_Head_act0 (LeakyReLU)         (None, 67, 120, 128) 0           G_Head_norm0[0][0]               
____________________________________________________________________________________________

In [ ]:
for e in range(10001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = (cv2.imread(test_low_path) / 255.0)[np.newaxis, :, :, :]
        test_img_high = (cv2.imread(test_high_path) / 255.0)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = (test_img_sr * 255).astype('u1')[0]
        test_out_path = test_out_dir + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:0.4914948344230652, d_loss_fake:1.0407686233520508, vgg_loss:0.5939614176750183
epoch 100
d_loss_real:0.5160975456237793, d_loss_fake:0.624387264251709, vgg_loss:0.5183566808700562
epoch 200
d_loss_real:0.6036903262138367, d_loss_fake:0.6694714426994324, vgg_loss:0.33328622579574585
epoch 300
d_loss_real:0.6745109558105469, d_loss_fake:0.702735185623169, vgg_loss:0.19668911397457123
epoch 400
d_loss_real:0.712861180305481, d_loss_fake:0.6736838817596436, vgg_loss:0.31766003370285034
epoch 500
d_loss_real:0.6955215930938721, d_loss_fake:0.6811579465866089, vgg_loss:0.14818622171878815
epoch 600
d_loss_real:0.6481887102127075, d_loss_fake:0.6986109018325806, vgg_loss:0.12946920096874237
epoch 700
d_loss_real:0.6616953611373901, d_loss_fake:0.6573883891105652, vgg_loss:0.14242161810398102
epoch 800
d_loss_real:0.6652133464813232, d_loss_fake:0.7414003610610962, vgg_loss:0.18427929282188416
epoch 900
d_loss_real:0.670425534248352, d_loss_fake:0.721577525138855, vgg_loss

In [ ]:
out_model_name = '20200604_generator.h5'
srgan.generator.save(model_dir + out_model_name, include_optimizer=False)